# Supervised Model

In [ ]:
from torch.utils.data import DataLoader
from torchvision import transforms
from progressBar import printProgressBar

import medicalDataLoader
import argparse
from utils import *

from UNet_Base import *
import random
import torch
import pdb

In [ ]:
import torch
print(torch.version.cuda)

# Select GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
def runTraining(epoch_num, weights_path='', augm = False):
    print('-' * 40)
    print('~~~~~~~~  Starting the training... ~~~~~~')
    print('-' * 40)

    ## DEFINE HYPERPARAMETERS (batch_size > 1)
    batch_size = 16
    batch_size_val = 16
    lr = 0.001   # Learning Rate
    epoch = epoch_num # Number of epochs
    
    root_dir = './Data/'

    print(' Dataset: {} '.format(root_dir))

    ## DEFINE THE TRANSFORMATIONS TO DO AND THE VARIABLES FOR TRAINING AND VALIDATION
    
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    mask_transform = transforms.Compose([
        transforms.ToTensor()
    ])

    train_set_full = medicalDataLoader.MedicalImageDataset('train',
                                                      root_dir,
                                                      transform=transform,
                                                      mask_transform=mask_transform,
                                                      augment=augm,  # Set to True to enable data augmentation
                                                      equalize=False)

    train_loader_full = DataLoader(train_set_full,
                              batch_size=batch_size,
                              worker_init_fn=np.random.seed(0),
                              num_workers=0,
                              shuffle=True)


    val_set = medicalDataLoader.MedicalImageDataset('val',
                                                    root_dir,
                                                    transform=transform,
                                                    mask_transform=mask_transform,
                                                    equalize=False)

    val_loader = DataLoader(val_set,
                            batch_size=batch_size_val,
                            worker_init_fn=np.random.seed(0),
                            num_workers=0,
                            shuffle=False)


    ## INITIALIZE YOUR MODEL
    num_classes = 4 # NUMBER OF CLASSES

    print("~~~~~~~~~~~ Creating the UNet model ~~~~~~~~~~")
    modelName = 'Test_Model'
    print(" Model Name: {}".format(modelName))

    ## CREATION OF YOUR MODEL
    net = UNet(num_classes)
    net = net.to(device)  # Move the model to the device

    ## Load the weights from the previously trained model
    if weights_path != '':
        # previous_model_dir = './models/' + 'Test_Model' + '/' + str(epoch_num) +'_Epoch'
        net.load_state_dict(torch.load(weights_path))
        print(" Model loaded: {}".format(weights_path))

    print("Total params: {0:,}".format(sum(p.numel() for p in net.parameters() if p.requires_grad)))

    # DEFINE YOUR OUTPUT COMPONENTS (e.g., SOFTMAX, LOSS FUNCTION, ETC)
    softMax = torch.nn.Softmax(dim=1)
    CE_loss = torch.nn.CrossEntropyLoss()

    ## PUT EVERYTHING IN GPU RESOURCES    
    if torch.cuda.is_available():
        net.cuda()
        softMax.cuda()
        CE_loss.cuda()

    ## DEFINE YOUR OPTIMIZER
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    ### To save statistics ####
    lossTotalTraining = []
    Best_loss_val = 1000
    BestEpoch = 0
    
    directory = 'Results/Statistics/' + modelName

    print("~~~~~~~~~~~ Starting the training ~~~~~~~~~~")
    if os.path.exists(directory)==False:
        os.makedirs(directory)

    ## START THE TRAINING
    
    ## FOR EACH EPOCH
    for i in range(epoch):
        net.train()
        lossEpoch = []
        DSCEpoch = []
        DSCEpoch_w = []
        num_batches = len(train_loader_full)
        
        ## FOR EACH BATCH
        for j, data in enumerate(train_loader_full):
            ### Set to zero all the gradients
            net.zero_grad()
            optimizer.zero_grad()

            ## GET IMAGES, LABELS and IMG NAMES
            images, labels, img_names = data            

            ### From numpy to torch variables
            labels = to_var(labels)
            images = to_var(images)

            ################### Train ###################
            #-- The CNN makes its predictions (forward pass)
            net_predictions = net(images)

            #-- Compute the losses --#
            # THIS FUNCTION IS TO CONVERT LABELS TO A FORMAT TO BE USED IN THIS CODE
            segmentation_classes = getTargetSegmentation(labels)
            
            # COMPUTE THE LOSS
            CE_loss_value = CE_loss(softMax(net_predictions), segmentation_classes) # XXXXXX and YYYYYYY are your inputs for the CE
            lossTotal = CE_loss_value

            # DO THE STEPS FOR BACKPROP (two things to be done in pytorch)
            lossTotal.backward()
            optimizer.step()

            # THIS IS JUST TO VISUALIZE THE TRAINING 
            lossEpoch.append(lossTotal.cpu().data.numpy())
            printProgressBar(j + 1, num_batches,
                             prefix="[Training] Epoch: {} ".format(i),
                             length=15,
                             suffix=" Loss: {:.4f}, ".format(lossTotal))

        lossEpoch = np.asarray(lossEpoch)
        lossEpoch = lossEpoch.mean()

        lossTotalTraining.append(lossEpoch)

        printProgressBar(num_batches, num_batches,
                             done="[Training] Epoch: {}, LossG: {:.4f}".format(i,lossEpoch) + '\n' 
                             + "[Validation] Epoch: {}, LossG: {:.4f}".format(i,lossEpoch))

        
        ## THIS IS HOW YOU WILL SAVE THE TRAINED MODELS AFTER EACH EPOCH. 
        ## WARNING!!!!! YOU DON'T WANT TO SAVE IT AT EACH EPOCH, BUT ONLY WHEN THE MODEL WORKS BEST ON THE VALIDATION SET!!
        if not os.path.exists('./models/' + modelName):
                os.makedirs('./models/' + modelName)

        torch.save(net.state_dict(), './models/' + modelName + '/' + str(i) + '_Epoch')
            
        np.save(os.path.join(directory, 'Losses.npy'), lossTotalTraining)


In [ ]:
runTraining(10000)

In [ ]:
# Run a training loop
# Looping from 9998 to 9994
for i in range(9998, 9994, -1):
    print("Epoch: {}".format(i))
    runTraining(i)


In [ ]:
# Testing the model on an Image

from PIL import Image
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt

def plotOutput(model_path, image_path):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    ## CREATION OF YOUR MODEL
    net_test = UNet(4)
    net_test = net_test.to(device)
    net_test.load_state_dict(torch.load(model_path))

    # Load the image
    image = Image.open(image_path)

    # Convert the image to grayscale
    if image.mode != 'L':
        image = image.convert('L')

    # Preprocess the image
    x = TF.to_tensor(image)
    x = TF.normalize(x, [0.5], [0.5])
    x = x.unsqueeze(0)  # Add batch dimension

    # Move the tensor to the same device as the model
    x = x.to(device)

    output = net_test(x)

    # The output is the predicted segmentation
    predicted_segmentation = torch.argmax(output.squeeze(), dim=0)

    # Convert the tensor to a numpy array
    predicted_segmentation = predicted_segmentation.detach().cpu().numpy()

    # Display the predicted segmentation
    plt.imshow(predicted_segmentation, cmap='gray')
    plt.show()

In [ ]:
# Output the training and validation loss of given weights

def getLoss(weights_path):
    # Load the weights from the file
    model = UNet(4)
    model.load_state_dict(torch.load(weights_path))

    # Determine the device and move the model to the device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # DEFINE YOUR OUTPUT COMPONENTS (e.g., SOFTMAX, LOSS FUNCTION, ETC)
    softMax = torch.nn.Softmax(dim=1)
    CE_loss = torch.nn.CrossEntropyLoss()

    # Set the model to evaluation mode
    model.eval()

    # Initialize the loss function
    CE_loss = torch.nn.CrossEntropyLoss()

    # Initialize a variable to store the total loss
    root_dir = './Data/'

    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    mask_transform = transforms.Compose([
        transforms.ToTensor()
    ])

    train_set_full = medicalDataLoader.MedicalImageDataset('train',
                                                        root_dir,
                                                        transform=transform,
                                                        mask_transform=mask_transform,
                                                        augment=True,  # Set to True to enable data augmentation
                                                        equalize=False)

    train_loader_full = DataLoader(train_set_full,
                                batch_size=16,
                                worker_init_fn=np.random.seed(0),
                                num_workers=0,
                                shuffle=True)


    val_set = medicalDataLoader.MedicalImageDataset('val',
                                                    root_dir,
                                                    transform=transform,
                                                    mask_transform=mask_transform,
                                                    equalize=False)

    val_loader = DataLoader(val_set,                 
                            batch_size=16,       
                            worker_init_fn=np.random.seed(0),
                            num_workers=0,
                            shuffle=False)

    lossEpoch_train = []
    lossEpoch_val = []


    for j, data in enumerate(train_loader_full):
        ### Set to zero all the gradients
        model.zero_grad()

        ## GET IMAGES, LABELS and IMG NAMES
        images, labels, img_names = data            

        ### From numpy to torch variables
        labels = to_var(labels)
        images = to_var(images)

        ################### Train ###################
        #-- The CNN makes its predictions (forward pass)
        net_predictions = model(images)

        #-- Compute the losses --#
        # THIS FUNCTION IS TO CONVERT LABELS TO A FORMAT TO BE USED IN THIS CODE
        segmentation_classes = getTargetSegmentation(labels)
        
        # COMPUTE THE LOSS
        CE_loss_value = CE_loss(softMax(net_predictions), segmentation_classes) # XXXXXX and YYYYYYY are your inputs for the CE
        
        lossTotal = CE_loss_value
        lossEpoch_train.append(lossTotal.item())

    lossEpoch_train = np.asarray(lossEpoch_train)
    lossEpoch_train = lossEpoch_train.mean()


    for j, data in enumerate(val_loader):
        ## GET IMAGES, LABELS and IMG NAMES
        images, labels, img_names = data            

        ### From numpy to torch variables
        labels = to_var(labels)
        images = to_var(images)

        ################### Validate ###################
        #-- The CNN makes its predictions (forward pass)
        net_predictions = model(images)

        #-- Compute the losses --#
        # THIS FUNCTION IS TO CONVERT LABELS TO A FORMAT TO BE USED IN THIS CODE
        segmentation_classes = getTargetSegmentation(labels)
        
        # COMPUTE THE LOSS
        CE_loss_value = CE_loss(softMax(net_predictions), segmentation_classes) # XXXXXX and YYYYYYY are your inputs for the CE
        
        lossTotal = CE_loss_value
        lossEpoch_val.append(lossTotal.item())

    lossEpoch_val = np.asarray(lossEpoch_val)
    lossEpoch_val = lossEpoch_val.mean()

    print('Training loss:', lossEpoch_train)
    print('Validation loss:', lossEpoch_val)
    return lossEpoch_train, lossEpoch_val

In [ ]:
# Plotting the training and validation loss for multiple models
import matplotlib.pyplot as plt

# models_paths = ['./models/' + 'Trained' + '/Adam_Labeled_1.3k_Epoch']
# models_paths = ['./models/' + 'Trained\Data_Augmented' + '/Adam_Labeled_' + str(i) + '5k_Epoch' for i in range(0, 8)]
models_paths = ['./models/' + 'Test_Model' + '/' + str(i) + '0_Epoch' for i in range(10,1500,100)]
# List to store the losses
train_losses = []
val_losses = []

# Iterate over the list of paths
for path in models_paths:
    # Get the losses for the current path
    train_loss, val_loss = getLoss(path)
    
    # Add the losses to the lists
    train_losses.append(train_loss)
    val_losses.append(val_loss)

# Plot the losses
epochs = range(len(train_losses))
# epochs = [(5 + 10*i)*1000 for i in range(len(train_losses))]

plt.figure(figsize=(12, 6))
plt.plot(epochs, train_losses, 'r', label='Training loss')
plt.plot(epochs, val_losses, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Displaying the outputs for different models

models_paths = ['./models/' + 'Test_Model' + '/' + str(i) + '999_Epoch' for i in range(1,15)]
# models_paths = ['./models/' + 'Trained/Data_Augmented_15K_EpochPlus' + '/Adam_Labeled_' + str(i) + '5k_Epoch' for i in range(0, 8)]
image_path = "./Data/val/Img/patient061_01_1.png"

for model_path in models_paths:
    plotOutput(model_path, image_path)

# Semi-Supervised

The objective is to implement a program based on this article [FixMatch](https://arxiv.org/abs/2001.07685). 

## Imports
Importing different Libraries to be able to treat data

In [5]:
from torch.utils.data import DataLoader
from torchvision import transforms
from progressBar import printProgressBar

import medicalDataLoader
import argparse
from utils import *

from UNet_Base import *
import random
import torch
import pdb

Which torch version is installed on the device

In [6]:
import torch
print(torch.version.cuda)

# Select GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

None
cpu


In [7]:
import warnings
warnings.filterwarnings("ignore")

## Training
We'll have one classic supervised model with no transformed image dataset. The results will be use as a a reference for the final model. The final model is built by comparing an none transformed image result to the result of the previous model and by comparing an output of this image and all transformed images based on this one and minimizing the difference.

This way the model will not base his comparaison on the position in the image but will be able to identify

### Teacher

The teacher is a basic supervised model where only labled data are beeing used to train the model

In [30]:
# Loading Data
def loadData(mode, batchSize, rootDir='./Data', _augment=False, _shuffle=False, _equalize=False,_numWorkers=0):
    print("~~~~~~~~~~~ Loading Data ~~~~~~~~~~")
    
    root_dir = './Data/'

    print(' Dataset: {} '.format(root_dir))

    ## DEFINE THE TRANSFORMATIONS TO DO AND THE VARIABLES FOR TRAINING AND VALIDATION
    
    _transform = transforms.Compose([
        transforms.ToTensor()
    ])

    _mask_transform = transforms.Compose([
        transforms.ToTensor()
    ])

    _set = medicalDataLoader.MedicalImageDataset(mode,
                                                      rootDir,
                                                      transform=_transform,
                                                      mask_transform=_mask_transform,
                                                      augment=_augment,  # Set to True to enable data augmentation
                                                      equalize=_equalize)

    loader = DataLoader(_set,
                              batch_size=batchSize,
                              worker_init_fn=np.random.seed(0),
                              num_workers=_numWorkers,
                              shuffle=_shuffle)

    return loader



In [31]:
def initModel(numberOfClasses, weights_path, modelName):
    ## INITIALIZE YOUR MODEL
    num_classes = 4 # NUMBER OF CLASSES

    print("~~~~~~~~~~~ Creating the UNet model ~~~~~~~~~~")
    print(" Model Name: {}".format(modelName))

    ## CREATION OF YOUR MODEL
    net = UNet(num_classes)
    net = net.to(device)  # Move the model to the device

        ## Load the weights from the previously trained model
    if weights_path != '':
        net.load_state_dict(torch.load(weights_path))
        print(" Model loaded: {}".format(weights_path))

    print("Total params: {0:,}".format(sum(p.numel() for p in net.parameters() if p.requires_grad)))
    return net

In [32]:
def gpuResources(net,softmax,CE_loss):
    ## PUT EVERYTHING IN GPU RESOURCES    
    if torch.cuda.is_available():
        net.cuda()
        softMax.cuda()
        CE_loss.cuda()

In [37]:
def runTraining(epoch_num, weights_path='', augm = False):
    print('-' * 40)
    print('~~~~~~~~  Starting the training... ~~~~~~')
    print('-' * 40)

    ## DEFINE HYPERPARAMETERS (batch_size > 1)
    batch_size = 16
    batch_size_val = 16
    lr = 0.001   # Learning Rate
    epoch = epoch_num # Number of epochs
    
    train_loader_full = loadData('train', batch_size,_augment=augm, _shuffle=True)
    val_loader = loadData('val', batch_size_val,_augment=augm)

    modelName = 'Test_Model'
    net = initModel(4, weights_path,modelName)

    # DEFINE YOUR OUTPUT COMPONENTS (e.g., SOFTMAX, LOSS FUNCTION, ETC)
    softmax = torch.nn.Softmax(dim=1)
    CE_loss = torch.nn.CrossEntropyLoss()

    gpuResources(net,softmax,CE_loss)

    ## DEFINE YOUR OPTIMIZER
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    ### To save statistics ####
    lossTotalTraining = []
    Best_loss_val = 1000
    BestEpoch = 0
    

    print("~~~~~~~~~~~ Starting the training ~~~~~~~~~~")
    # Create saving directory if not already existing
    directory = 'Results/Statistics/' + modelName
    if os.path.exists(directory)==False:
        os.makedirs(directory)

    ## START THE TRAINING
    
    ## FOR EACH EPOCH
    for i in range(epoch):
        net.train()
        lossEpoch = []
        DSCEpoch = []
        DSCEpoch_w = []
        num_batches = len(train_loader_full)
        
        ## FOR EACH BATCH
        for j, data in enumerate(train_loader_full):
            ### Set to zero all the gradients
            net.zero_grad()
            optimizer.zero_grad()

            ## GET IMAGES, LABELS and IMG NAMES
            images, labels, img_names = data            

            ### From numpy to torch variables
            labels = to_var(labels)
            images = to_var(images)

            ################### Train ###################
            #-- The CNN makes its predictions (forward pass)
            net_predictions = net(images)

            #-- Compute the losses --#
            # THIS FUNCTION IS TO CONVERT LABELS TO A FORMAT TO BE USED IN THIS CODE
            segmentation_classes = getTargetSegmentation(labels)
            
            # COMPUTE THE LOSS
            CE_loss_value = CE_loss(softmax(net_predictions), segmentation_classes) # XXXXXX and YYYYYYY are your inputs for the CE
            lossTotal = CE_loss_value

            # DO THE STEPS FOR BACKPROP (two things to be done in pytorch)
            lossTotal.backward()
            optimizer.step()

            # THIS IS JUST TO VISUALIZE THE TRAINING 
            lossEpoch.append(lossTotal.cpu().data.numpy())
            printProgressBar(j + 1, num_batches,
                             prefix="[Training] Epoch: {} ".format(i),
                             length=15,
                             suffix=" Loss: {:.4f}, ".format(lossTotal))

        lossEpoch = np.asarray(lossEpoch)
        lossEpoch = lossEpoch.mean()

        lossTotalTraining.append(lossEpoch)

        printProgressBar(num_batches, num_batches,
                             done="[Training] Epoch: {}, LossG: {:.4f}".format(i,lossEpoch) + '\n' 
                             + "[Validation] Epoch: {}, LossG: {:.4f}".format(i,lossEpoch))

        
        ## THIS IS HOW YOU WILL SAVE THE TRAINED MODELS AFTER EACH EPOCH. 
        ## WARNING!!!!! YOU DON'T WANT TO SAVE IT AT EACH EPOCH, BUT ONLY WHEN THE MODEL WORKS BEST ON THE VALIDATION SET!!
        if not os.path.exists('./models/' + modelName):
                os.makedirs('./models/' + modelName)

        torch.save(net.state_dict(), './models/' + modelName + '/' + str(i) + '_Epoch')
            
        np.save(os.path.join(directory, 'Losses.npy'), lossTotalTraining)

    return os.path.join(directory, 'Losses.npy')


In [39]:
runTraining(10)

----------------------------------------
~~~~~~~~  Starting the training... ~~~~~~
----------------------------------------
~~~~~~~~~~~ Loading Data ~~~~~~~~~~
 Dataset: ./Data/ 
~~~~~~~~~~~ Loading Data ~~~~~~~~~~
 Dataset: ./Data/ 
~~~~~~~~~~~ Creating the UNet model ~~~~~~~~~~
 Model Name: Test_Model
Total params: 60,664
~~~~~~~~~~~ Starting the training ~~~~~~~~~~
[Training] Epoch: 0 [DONE]                                 
[Training] Epoch: 0, LossG: 1.4694===================================================================] 100.0%
[Validation] Epoch: 0, LossG: 1.4694                                      
[Training] Epoch: 1 [DONE]                                 
[Training] Epoch: 1, LossG: 1.4414===================================================================] 100.0%
[Validation] Epoch: 1, LossG: 1.4414                                      
[Training] Epoch: 2 [DONE]                                 
[Training] Epoch: 2, LossG: 1.4143=============================================

KeyboardInterrupt: 